In [ ]:
!pip install crewai crewai_tools langchain_community python-dotenv

In [78]:
from dotenv import load_dotenv
load_dotenv()

True

In [79]:
from langchain_community.llms import Ollama
from crewai import Agent, Task, Crew, Process
from crewai_tools import DirectorySearchTool, SerperDevTool

class LegalAnalysisCrew:
    def __init__(self, model_name="gemma3:1b"):
        self.model_name = model_name
        
        self.rag_tool = DirectorySearchTool(
            directory="resources/",
            config=dict(
                llm=dict(
                    provider="ollama",
                    config=dict(
                        model=model_name,
                    ),
                ),
                embedder=dict(
                    provider="google",
                    config=dict(
                        model="models/embedding-001",
                        task_type="retrieval_document",
                    ),
                ),
            )
        )

        self.ollama_llm = Ollama(model=self.model_name)
        
        self.search_tool = SerperDevTool(country = "br")
        self._define_agents()

    def _define_agents(self):

        self.pesquisador_legal = Agent(
            role='Pesquisador Jurídico Especialista',
            goal=f'Analisar a pergunta do usuário, identificar as áreas legais relevantes (Consumidor, Civil, Proteção de Dados) e usar a ferramenta RAG para encontrar os artigos de lei exatos que se aplicam ao caso.',
            backstory='Você é um advogado meticuloso com vasta experiência em pesquisa jurídica. Sua principal habilidade é conectar problemas do mundo real com o texto exato da lei.',
            verbose=True,
            llm=self.ollama_llm,
            tools=[self.rag_tool, self.search_tool],
            allow_delegation=False
        )

        self.analista_de_contexto = Agent(
            role='Analista de Contexto e Casos Práticos',
            goal=f'Para qualquer pergunta do usuário, usar a busca na web para encontrar notícias, artigos de blog e decisões judiciais que forneçam contexto do mundo real sobre como as leis são aplicadas.',
            backstory='Você é um jornalista investigativo especializado no judiciário. Você sabe como encontrar exemplos que ilustram a aplicação prática das leis, ajudando a entender além do texto legal.',
            verbose=True,
            llm=self.ollama_llm,
            tools=[self.search_tool],
            allow_delegation=False
        )

        self.revisor_sintetizador = Agent(
            role='Revisor Jurídico Sênior',
            goal=f'Revisar as análises dos pesquisadores, comparar as informações da lei com o contexto da web, e sintetizar tudo em uma resposta final, coesa, bem-estruturada e fácil de entender para o usuário.',
            backstory='Com anos de experiência como juiz, você é especialista em consolidar múltiplas fontes de informação em uma única conclusão lógica e bem fundamentada. Sua clareza e precisão são incomparáveis.',
            verbose=True,
            llm=self.ollama_llm,
            allow_delegation=False
        )

    def run(self, question: str):
        """Executa o fluxo de trabalho da crew para uma determinada pergunta."""
        
        task_pesquisa_legal = Task(
            description=f"Analise a pergunta do usuário: '{question}'. Foque em identificar e extrair os artigos relevantes da sua base de conhecimento legal (RAG tool) que se aplicam diretamente a esta situação.",
            expected_output="Uma lista de artigos de lei (do CDC, LGPD, Código Civil) relevantes para a pergunta, com uma breve explicação de cada um.",
            agent=self.pesquisador_legal,
            async_execution=True
        )

        task_analise_contexto = Task(
            description=f"Pesquise na web por contexto prático relacionado à pergunta do usuário: '{question}'. Encontre notícias, posts de blogs de advogados ou resumos de casos que mostrem como essa situação é tratada na prática.",
            expected_output="Um resumo das descobertas da web, incluindo links para 2-3 fontes importantes que dão contexto sobre o tema.",
            agent=self.analista_de_contexto,
            async_execution=True
        )

        task_revisao_final = Task(
            description=f"Usando os resultados da pesquisa legal e da análise de contexto, formule uma resposta final abrangente e bem estruturada para a pergunta do usuário: '{question}'. Sua resposta deve: 1. Explicar claramente os direitos e deveres com base nos artigos de lei encontrados. 2. Usar o contexto da web para dar exemplos práticos. 3. Ser dividida em seções lógicas para fácil leitura. 4. Concluir com um resumo dos próximos passos que o usuário pode tomar e o disclaimer legal.",
            expected_output="A resposta final e completa a ser entregue ao usuário.",
            agent=self.revisor_sintetizador,
            context=[task_pesquisa_legal, task_analise_contexto]
        )

        # Montar e executar a crew com as tarefas dinâmicas
        crew = Crew(
            agents=[self.pesquisador_legal, self.analista_de_contexto, self.revisor_sintetizador],
            tasks=[task_pesquisa_legal, task_analise_contexto, task_revisao_final],
            process=Process.sequential,
            verbose=2
        )

        return crew.kickoff()

In [80]:
print("Criando instância da LegalAnalysisCrew...")
legal_crew = LegalAnalysisCrew()

Criando instância da LegalAnalysisCrew...


/home/castro/.local/share/virtualenvs/agents-3pPokBLJ/lib/python3.12/site-packages/alembic/config.py:592: DeprecationWarning: No path_separator found in configuration; falling back to legacy splitting on spaces, commas, and colons for prepend_sys_path.  Consider adding path_separator=os to Alembic config.
  util.warn_deprecated(
Inserting batches in chromadb:   0%|          | 0/18 [00:12<?, ?it/s]